In [1]:
import requests
from csv import writer
from pprint import pprint
from bs4 import BeautifulSoup
import shutil
import pandas as pd
import cv2
import numpy as np
import os
from datetime import datetime, timedelta
import datetime

In [2]:
mug_details = pd.read_csv('../data/mugshot_details.csv')
mug_individuals = pd.read_csv('../data/individual_details.csv')
existing_id = list(mug_details.booking_number)

In [3]:
def save_mugshot(booking_id,url,img_path):

    resp = requests.get(url, stream=True)
    local_file = open(f'{img_path}{booking_id}.png', 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)


def append_csv(file,data):
    with open(f'../data/{file}.csv', 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(data)

        
def remove_blanks(img_path):
    for img in os.listdir(img_path):

        path = os.path.join(img_path, img)
        filename = img.split('.')[0]
        img_array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if np.shape(img_array) == ():
            os.remove(img_path + img)
        
        
def get_individual(booking_id):
    URL = f'https://www.mcso.org/Mugshot/GetMugshotDetails?bookingNumber={booking_id}'
    booking_url = requests.get(URL)
    booking_page = BeautifulSoup(booking_url.content, 'html.parser')

    details_match = booking_page.findAll('div', class_='col-md-6')[1].text
    piece = details_match.split('\n')

    details = []

    for p in piece:
        if p not in '':
            details.append(p.split(':')[-1].strip())


    crime_match = booking_page.find('div', class_='col-md-9')

    crimes = []

    for m in crime_match.findAll('div'):
        count = m.text.split('of ')[1].rstrip()

        if count not in '':
            if count not in crimes:
                crimes.append(count)

    details.append(crimes)

    return details

# Maricopa County

In [4]:
URL = 'https://www.mcso.org/Mugshot'
page = requests.get(URL)
main = BeautifulSoup(page.content, 'html.parser')
img_path = '../mugshots/mugs/'

In [5]:
url_index = 0
index = 0

bookings = main.findAll('td', attrs={"style": "vertical-align: middle"})

booking_detail = []
individual_details = []

while index <= 1326:

    individual_detail = []

    # parse the individual details
    for i in range(index,index+6):
        individual_detail.append(bookings[i].text.rstrip())

    # Want to stop when an existing record is found
    if individual_detail[0] in existing_id:
        break

    # Append individual mughshot details
    append_csv('../data/mugshot_details', individual_detail)

    # Mugshot downlaod
    image_url = f'https://www.mcso.org/Images/Mugshots/{individual_detail[0]}_030000.jpg'
    save_mugshot(individual_detail[0],image_url,img_path)

    # Append individual booking details
    append_csv('individual_details', get_individual(individual_detail[0]))

    # Update indexes to parse html
    url_index += 1
    index += 6

In [6]:
remove_blanks(img_path)

# Osceola County

In [7]:
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)
booking_date_string = yesterday.strftime('%Y-%m-%d')

In [8]:
img_path = '../mugshots/osceola/'
downloaded_images = [img.split('.')[0] for img in os.listdir(img_path)]

In [9]:
url = f'https://apps.osceola.org/Apps/CorrectionsReports/Report/Daily/{booking_date_string}'
page = requests.get(url)
main = BeautifulSoup(page.content, 'html.parser')

bookings = main.findAll('span', class_="arrest-number")

booking_numbers = [(str(b).split('>')[1].split('<')[0]) for b in bookings]

for inmate_id in booking_numbers:

    if inmate_id not in downloaded_images:

        image_url = f'http://198.140.240.14/files/modules/Corrections/InmateSearch/mugshot/{inmate_id}.jpg'

        save_mugshot(inmate_id,image_url,img_path)

In [10]:
remove_blanks(img_path)